In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression

# Загрузка данных
path = "DataSets/dataframe_benign_brute_xss_sql.csv"
df = pd.read_csv(path)
df.columns = [col.lstrip() for col in df.columns]

# Преобразование меток классов в числа
label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])

# Выделение признаков и меток
X = df.drop('Label', axis=1)
y = df['Label']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

# Андерсэмплинг для балансировки классов
under_sampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_train)

# Обучение модели
# Вы можете выбрать любую модель, здесь пример с LogisticRegression
# clf = LogisticRegression(max_iter=1000)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_resampled, y_resampled)

# Предсказание на тестовых данных
y_pred = clf.predict(X_test)



In [7]:
# Расчет точности и вывод отчета по классификации
accuracy = accuracy_score(y_test, y_pred)
print("Точность:", accuracy)
print("Отчет по классификации:\n", classification_report(y_test, y_pred))

Точность: 0.8368859200080563
Отчет по классификации:
               precision    recall  f1-score   support

           0       1.00      0.84      0.91    117633
           1       0.09      0.47      0.15      1061
           2       0.00      1.00      0.00        13
           3       0.10      0.55      0.16       455

    accuracy                           0.84    119162
   macro avg       0.30      0.72      0.31    119162
weighted avg       0.99      0.84      0.90    119162

